In [1]:
import numpy as np

from scipy.signal import fftconvolve
from scipy.interpolate import interp1d
from scipy.optimize import minimize

from pulse2percept import electrode2currentmap as e2cm
from pulse2percept import utils

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Data
---------

In [2]:
# Data extracted from Fig. 3a in Jensen et al. (2005)
jensen2005 = dict()
jensen2005['E125'] = dict()
jensen2005['E500'] = dict()

# Axons
jensen2005['E125']['axon'] = np.array([0.10000, 2.99614, 0.19835, 1.59634, 0.49824,
                                   0.72339, 1.00000, 0.54247, 2.00708, 0.48697,
                                   5.04155, 0.46555, 10.00000, 0.46976, 20.30918,
                                   0.46976, 50.41550, 0.46555]).reshape((-1, 2))
jensen2005['E125']['axon'][:, 0] /= 1000.0  # convert ms to s

jensen2005['E500']['axon'] = np.array([0.09883, 20.86371, 0.19535, 8.69288, 0.48840,
                                   4.50809, 0.98832, 2.93549, 1.90815, 2.31750,
                                   4.88400, 2.19890, 9.54095, 2.08637, 19.08149,
                                   2.06818, 47.70583, 1.99701]).reshape((-1, 2))
jensen2005['E500']['axon'][:, 0] /= 1000.0  # from ms to s


# SL cells
# The SL cell responses were typically a single action potential, although a few cells elicited
# a second action potential that immediately followed the first.
jensen2005['E125']['SL'] = np.array([0.10239, 0.74318, 0.19372, 0.62082, 0.49824,
                                 0.46976, 1.00000, 0.37855, 1.96023, 0.39954,
                                 5.04155, 0.40679, 10.00000, 0.39954, 19.83517,
                                 0.39242, 49.82369, 0.35866]).reshape((-1, 2))
jensen2005['E125']['SL'][:, 0] /= 1000.0  # from ms to s

jensen2005['E500']['SL'] = np.array([0.09883, 6.86281, 0.20000, 4.50809, 0.47706,
                                 2.83447, 0.98832, 2.01457, 1.93070, 2.05016,
                                 4.77058, 1.99701, 9.54095, 2.01457, 19.08149,
                                 1.97960, 47.14866, 1.87830]).reshape((-1, 2))
jensen2005['E500']['SL'][:, 0] /= 1000.0  # from ms to s


# LL cells
# The LL cell responses consisted of one or more bursts of typically two to five action potentials
# after stimulation at or slightly above threshold current. Unlike the SL cell responses, the
# LL cell responses did not appear to be due to direct activation of RGCs but rather to
# presynaptic activation of retinal neurons.
# LL (delayed) responses have higher thresholds than SL (immediate) responses.
jensen2005['E125']['LL'] = np.array([0.09767, 15.26138, 0.19835, 9.13982, 0.49824,
                                 4.78286, 0.98826, 3.02321, 1.96023, 2.01691,
                                 4.92388, 0.93057, 10.00000, 0.67317, 19.60233,
                                 0.35227, 49.82369, 0.28133]).reshape((-1, 2))
jensen2005['E125']['LL'][:, 0] /= 1000.0  # from ms to s

jensen2005['E500']['LL'] = np.array([0.09883, 53.70788, 0.20000, 36.53740, 0.48840, 
                                 12.23073, 0.96537, 8.76932, 1.90815, 6.39858,
                                 4.88400, 4.13019, 9.65370, 2.46397, 19.08149,
                                 1.81366, 47.70583, 0.96559]).reshape((-1, 2))
jensen2005['E500']['LL'][:, 0] /= 1000.0  # from ms to s

In [3]:
# Data extracted from Fig. 4

# Cathodic
jensen2005['E125']['cathodic'] = np.array([0.09790, 3.60377, 0.19721, 1.80616, 0.50172,
                                       0.83488, 0.98945, 0.65090, 1.99315, 0.60031,
                                       4.96424, 0.58921, 9.89445, 0.58921, 20.14408,
                                       0.58555, 51.24806, 0.58555]).reshape((-1, 2))
jensen2005['E125']['cathodic'][:, 0] /= 1000

jensen2005['E500']['cathodic'] = np.array([0.09674, 12.99098, 0.19533, 7.89559, 0.49841,
                                       4.14732, 0.98500, 2.61376, 1.98677, 1.97357,
                                       4.94956, 1.87169, 9.86591, 1.84259, 19.66889,
                                       1.74952, 48.99084, 1.73072]).reshape((-1, 2))
jensen2005['E500']['cathodic'][:, 0] /= 1000



# Anodic
jensen2005['E125']['anodic'] = np.array([0.10000, 8.94028, 0.19931, 5.57116, 0.50172,
                                     3.83516, 1.01067, 3.42874, 2.03590, 3.10379,
                                     5.01719, 3.14266, 10.21449, 3.12316, 20.35897,
                                     3.06540, 51.24806, 3.18202]).reshape((-1, 2))
jensen2005['E125']['anodic'][:, 0] /= 1000

jensen2005['E500']['anodic'] = np.array([0.09849, 26.77548, 0.19485, 13.42013, 0.49741, 6.40109, 0.98211, 4.95176,
                              1.93901, 3.87704, 4.93281, 3.90493, 9.83307, 3.79816, 20.02002, 3.76131,
                              50.94491, 3.56675]).reshape((-1, 2))
jensen2005['E500']['anodic'][:, 0] /= 1000

In [4]:
# Data extracted from Fig. 2
jensen2007 = dict()

jensen2007['amp2amp1'] = np.array([0.98021, 0.06501, 1.96095, 0.22681,
                                   2.84399, 0.60933, 3.79513, 0.61970,
                                   4.63264, 0.68464, 5.43022, 0.78494,
                                   6.42094, 0.86290, 7.32954, 0.93068,
                                   8.28541, 1.00133]).reshape((-1, 2))
jensen2007['amp2amp1'][:, 0] = 1000.0 / np.array([15, 25, 40, 65, 100, 150, 250, 400, 650])

jensen2007['norm_resp'] = dict()
jensen2007['norm_resp']['1.5'] = np.array([1.11538, 0.99233, 2.13305, 0.99832,
                                           3.13831, 1.02989, 4.15798, 1.04791,
                                           5.13079, 0.98479, 6.14069, 0.94417,
                                           7.15497, 1.02987, 8.15510, 0.93062,
                                           9.17001, 0.92007, 10.19143, 0.94861]).reshape((-1, 2))
jensen2007['norm_resp']['2.5'] = np.array([1.13180, 0.99080, 2.14320, 0.95921,
                                           3.15460, 0.92761, 4.13668, 0.92012,
                                           5.14370, 0.96221, 6.17201, 0.93210,
                                           7.14858, 0.89153, 8.15184, 0.91107,
                                           9.14658, 0.87950, 10.18642, 0.91854]).reshape((-1, 2))
jensen2007['norm_resp']['4.0'] = np.array([1.11513, 0.99083, 2.11476, 0.88857,
                                           3.13192, 0.89156, 4.13781, 0.82689,
                                           5.09947, 0.79687, 6.16888, 0.81330,
                                           7.15397, 0.82385, 8.16349, 0.88098,
                                           9.13029, 0.78178, 10.18454, 0.80727]).reshape((-1, 2))
jensen2007['norm_resp']['6.5'] = np.array([1.14871, 0.99228, 2.10348, 0.82092,
                                           3.11313, 0.77880, 4.10010, 0.70063,
                                           5.12152, 0.72917, 6.13944, 0.73666,
                                           7.09584, 0.67507, 8.15184, 0.71107,
                                           9.13969, 0.73816, 10.14182, 0.65094]).reshape((-1, 2))
jensen2007['norm_resp']['10.0'] = np.array([1.14871, 0.99228, 2.12528, 0.75172,
                                            3.11000, 0.66001, 4.08294, 0.69764,
                                            5.12290, 0.63743, 6.12415, 0.64495,
                                            7.12365, 0.64194, 8.12065, 0.62390,
                                            9.10699, 0.64197, 10.12854, 0.57126]).reshape((-1, 2))
jensen2007['norm_resp']['15.0'] = np.array([1.13180, 0.99080, 2.08056, 0.58336,
                                            3.07204, 0.53225, 4.05700, 0.44204,
                                            5.08168, 0.49013, 6.07617, 0.45705,
                                            7.07141, 0.42849, 8.07930, 0.37585,
                                            9.04924, 0.39544, 10.08231, 0.39389]).reshape((-1, 2))
jensen2007['norm_resp']['25.0'] = np.array([1.11488, 0.98933, 2.08081, 0.58487,
                                            3.06176, 0.47061, 4.06703, 0.50218,
                                            5.09985, 0.49912, 6.05988, 0.35933,
                                            7.06139, 0.36835, 8.06640, 0.39842,
                                            9.05888, 0.35332, 10.08131, 0.38787]).reshape((-1, 2))
jensen2007['norm_resp']['40.0'] = np.array([1.11463, 0.98782, 2.00100, 0.20604,
                                            3.02694, 0.46164, 4.01090, 0.16542,
                                            5.06590, 0.39541, 5.98547, 0.11283,
                                            7.07880, 0.37284, 8.03170, 0.09020,
                                            9.06502, 0.29015, 10.04610, 0.07665]).reshape((-1, 2))
jensen2007['norm_resp']['65.0'] = np.array([1.13205, 0.99231, 1.99261, 0.05568,
                                            3.01002, 0.06016, 3.98910, 0.03463,
                                            5.03069, 0.08419, 6.00852, 0.05114,
                                            7.02180, 0.13082, 8.00000, 0.00003,
                                            9.00601, 0.03611, 10.01942, 0.01654]).reshape((-1, 2))

In [5]:
# Data extracted from Fig. 1
cai2011 = dict()

cai2011['100'] = np.array([0.46257, 0.00105, 0.97908, 0.13688, 1.92488, 1.00413,
                           2.93725, 1.00626, 3.94948, 1.01050, 4.92435, 1.01255,
                           5.93659, 1.01679, 6.92409, 1.01676, 7.94909, 1.01681]).reshape((-1, 2))
cai2011['200'] = np.array([0.47507, 0.00108, 0.95816, 0.07157, 1.95514, 0.71998,
                           2.93725, 1.00626, 3.93698, 1.01047, 4.92448, 1.01045,
                           5.94922, 1.01471, 6.93646, 1.01889, 7.93672, 1.01468]).reshape((-1, 2))
cai2011['300'] = np.array([0.47507, 0.00108, 0.99671, 0.05481, 1.95487, 0.52419,
                           2.93225, 0.88625, 3.93856, 0.98521, 4.94948, 1.01050,
                           5.93672, 1.01468, 6.93672, 1.01468, 7.94909, 1.01681]).reshape((-1, 2))

# Model
-----------------

Following are all utily funcions used in this script.

In [6]:
def gaussian(t, alpha, tau, t0=0, n=1):
    return alpha * ((t - t0) / tau) ** n * np.exp(-n * ((t - t0) / tau - 1))

def rgc_impulse(ampl_pos, ampl_min, tau_pos=0.817, tau_min=0.433, n=15, tsample=0.005):
    # TODO: make it work with seconds instead of milliseconds
    t = np.arange(0, 2.0, tsample)
    y = gaussian(t, ampl_pos, tau_pos, 0, n) - gaussian(t, ampl_min, tau_min, 0, n)
    return y

In [7]:
class GanglionCell(object):
    def __init__(self, tsample, ca_scale=1.0, ca_tau=0.0191,
                 inl_scale=0, inl_tau=5.0 / 1000, anodic_scale=0,
                 sig_a=3.0, sig_b=8.0):
        ca_scale > 0
        inl_scale > 0
        anodic_scale > 0
        ca_tau > 0
        self.tsample = tsample
        self.ca_scale = ca_scale
        self.ca_tau = ca_tau
        self.inl_scale = inl_scale
#         self.anodic_scale = anodic_scale
        self.sig_a = sig_a
        self.sig_b = sig_b

        self.gamma_gcl = rgc_impulse(anodic_scale, 1.0, tsample=tsample * 1000)
        self.gamma_gcl /= np.trapz(np.abs(self.gamma_gcl), dx=tsample)
        
        self.inl_tau = inl_tau
        self.delay_inl = 40.0 / 1000
        t = np.arange(0, 10 * self.inl_tau, self.tsample)
        tmp_gamma = e2cm.gamma(2, 10 * self.inl_tau, t)
        tmp_gamma /= np.trapz(np.abs(tmp_gamma), dx=tsample)
        delay = np.zeros(int(np.round(self.delay_inl / self.tsample)))
        t = np.concatenate((delay, t))
        self.gamma_inl = np.concatenate((delay, tmp_gamma))

        t = np.arange(0, 6 * self.ca_tau, self.tsample)
        self.gamma_ca = e2cm.gamma(1, self.ca_tau, t)
        self.gamma_ca /= np.trapz(self.gamma_ca, dx=self.tsample)

        
    def model_cascade(self, pt, dolayers=None, dojit=True):
        assert pt.tsample == self.tsample
        
        # Calculate charge accumulation on electrode
        self.stim = pt
        # TODO: what if charge accum grows slower for anodic
        self.ca = pt.tsample * fftconvolve(np.abs(pt.data), self.gamma_ca,
                                           mode='full')[:pt.data.size]

        # Calculate impulse responses for GCL and INL layers
        self.r_gcl_lin = utils.sparseconv(self.gamma_gcl, pt.data, mode='full', dojit=True)
        self.r_gcl_lin = pt.tsample * self.r_gcl_lin[:pt.data.size]

        self.r_inl_lin = utils.sparseconv(self.gamma_inl, pt.data, mode='full', dojit=True)
        self.r_inl_lin = pt.tsample * self.r_inl_lin[:pt.data.size]
        self.r_inl_lin = fftconvolve(self.gamma_gcl, self.r_inl_lin, mode='full')
        self.r_inl_lin = pt.tsample * self.r_inl_lin[:pt.data.size]
        
        self.r_lin = self.r_gcl_lin + self.inl_scale * self.r_inl_lin

        # Adjust for accumulated charge
        self.r_adj = self.r_lin - self.ca_scale * self.ca
        
        # Feed through logistic activation function
        self.r_rgc = 100.0 / (1.0 + np.exp(-self.sig_a * self.r_adj + self.sig_b))
        
        return utils.TimeSeries(pt.tsample, self.r_rgc)

Helper
------

In [8]:
def fit_amps(x, y, match_y):
    x = np.array(x)
    y = np.array(y)
    # We are given a list of input values `x` and their corresponding output values
    # `y` = f(x). We want to find the `x` value for which f(x) = `match_y`.
    if np.allclose(y.ptp(), 0.0):
        return np.inf

    # Easiest way is to use interp1d as x = f(y)!
    f = interp1d(y.flatten(), x.flatten(), fill_value='extrapolate')
    return np.asscalar(f(match_y))

In [9]:
# Monophasic pulse train
def get_jensen2005_pt(ptype, freq, amp, pdur, stim_dur, tsample=0.005 / 1000):
    assert freq >= 0
    assert amp >= 0  # sign handled by mono pulse
    assert pdur > 0
    assert stim_dur >= 0
    
    delay_dur = 20.0 / 1000
    
    stim_size = int(round(stim_dur / tsample))
    if np.isclose(amp, 0.0) or np.isclose(freq, 0.0):
        return utils.TimeSeries(tsample, np.zeros(stim_size))
    
    envelope_dur = 1.0 / float(freq)
    assert envelope_dur > delay_dur
    pulse_train = np.array([])
    for j in range(int(np.ceil(stim_dur * freq))):
        # A single mono pulse
        pulse = e2cm.get_monophasic_pulse(ptype, pdur, tsample, delay_dur, envelope_dur) * amp
        pulse_train = np.concatenate((pulse_train, pulse), axis=0)
    pulse_train = pulse_train[:stim_size]
    
    return utils.TimeSeries(tsample, pulse_train)

In [10]:
def find_sl_ll(resp, stim, delay_inl, verbose=False):
    """Finds the SL and LL responses of a pulse train"""
    
    # Find stimulus on-set in stimulus
    idx_onset = np.where(np.abs(stim.data) == np.abs(stim.data).max())[0][0]
    dur_onset = idx_onset * stim.tsample
    
    # Translate to indexing of `resp` (might have different `tsample`)
    idx_onset = int(np.floor(dur_onset / resp.tsample))
    
    if verbose:
        print('stim onset: dur=%f, idx=%d' % (dur_onset, idx_onset))
        
    activity = resp.data.squeeze()
    
    # SL response within 10ms of stimulus onset
    sl_dur = 10.0 / 1000
    sl_size = int(np.round(sl_dur / resp.tsample))
    sl_resp = activity[idx_onset: idx_onset + sl_size]
    t_sl = (sl_resp.argmax() + idx_onset) * resp.tsample
    y_sl = sl_resp.max()
    
    # LL response after ~40ms of stimulus onset
    ll_delay = int(np.floor(delay_inl / resp.tsample))
    ll_resp = activity[idx_onset + ll_delay:]
    t_ll = (ll_resp.argmax() + idx_onset + ll_delay) * resp.tsample
    y_ll = ll_resp.max()
    
    if verbose:
        print('SL=%f at t=%f, LL=%f at t=%f' % (y_sl, t_sl, y_ll, t_ll))
        
    return [t_sl, y_sl], [t_ll, y_ll]

# Fit
----------

In [11]:
def run_jensen2005_fig3(model, rtype, amp_th,
                        pulse_durs=np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50]) / 1000):
    stim_dur = 500.0 / 1000
    stim_freq = 2.0
    
    gcr = model['gcr']
    tsample = gcr.tsample
    
    fig3 = dict()
    fig3['pulse_durs'] = pulse_durs
    
    # We need to define some activity value that corresponds to threshold activity.
    # We do this by executing the model at the first data point of the Jensen paper.
    # Whatever activity is produced by these parameters, we'll take as threshold
    # activity, and will try to find the matching current for all subsequent data points
    stim = get_jensen2005_pt('cathodic', stim_freq, amp_th, pulse_durs[0], stim_dur, tsample)
    mov = gcr.model_cascade(stim)
    sl_max, ll_max = find_sl_ll(mov, stim, gcr.delay_inl)
    if rtype == 'SL':
        act_match = sl_max[1]
    else:
        act_match = ll_max[1]
    
    fig3['act_match'] = act_match

    amp_in = np.array([0.01, 0.05, 0.1, 0.5, 1.0]) * amp_th
    fig3['amp_in'] = amp_in
    

    amp_match = []
    for pd in pulse_durs:
        cond = '%.3f' % pd
        fig3[cond] = dict()

        act_out = []
        for amp in amp_in:
            stim = get_jensen2005_pt('cathodic', stim_freq, amp, pd, stim_dur, tsample)
            mov = gcr.model_cascade(stim)
            sl_max, ll_max = find_sl_ll(mov, stim, gcr.delay_inl)
            if rtype == 'SL':
                act_out.append(sl_max[1])
            else:
                act_out.append(ll_max[1])
                
        fig3[cond]['act_out'] = act_out

        f_sl = fit_amps(amp_in, act_out, act_match)
        if np.isnan(f_sl):
            print('nan for SL=%f, sl_out=%s' % (act_sl_match, [('%.3f' % s) for s in sl_out]))
            f_sl = np.inf
            
        amp_match.append(f_sl)
        
    fig3['amp_match'] = np.array(amp_match)
    return fig3

In [12]:
def run_jensen2005_fig4(model, ptype, amp_th,
                        pulse_durs=np.array([0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50]) / 1000,
                        verbose=False):
    stim_dur = 500.0 / 1000
    stim_freq = 2.0
    
    gcr = model['gcr']
    tsample = gcr.tsample
    
    fig4 = dict()
    fig4['pulse_durs'] = pulse_durs
    
    # We need to define some activity value that corresponds to threshold activity.
    # We do this by executing the model at the first data point of the Jensen paper.
    # Whatever activity is produced by these parameters, we'll take as threshold
    # activity, and will try to find the matching current for all subsequent data points
    stim = get_jensen2005_pt(ptype, stim_freq, amp_th, pulse_durs[0], stim_dur, tsample)
    mov = gcr.model_cascade(stim)
    act_match = mov.data.max()
    amp_in = np.array([0.05, 0.1, 0.5, 1.0]) * amp_th
    fig4['amp_in'] = amp_in

    if verbose:
        print('- Threshold=%f' %  act_match)

    amp_match = []
    for pd in pulse_durs:
        cond = '%.3f' % pd
        fig4[cond] = dict()

        if verbose:
            print('- Processing pulse dur %f' % pd)
        act_out = []
        for amp in amp_in:
            stim = get_jensen2005_pt(ptype, stim_freq, amp, pd, stim_dur, tsample)
            idx_onset = stim.data.nonzero()[0][0]
            mov = gcr.model_cascade(stim)
            act_out.append(mov.data.max())
        fig4[cond]['act_out'] = act_out
        amp_match.append(fit_amps(amp_in, act_out, act_match))
        
    fig4['amp_match'] = np.array(amp_match)
    return fig4

In [13]:
def run_jensen2007_trial(gcr, amp, freq, pdur=1.0 / 1000, idur=0.5 / 1000, num_pulses=1):
    # Finding threshold
    stim = e2cm.Psycho2Pulsetrain(gcr.tsample, freq=freq, amp=amp, dur=float(num_pulses) / freq,
                                  pulse_dur=pdur, interphase_dur=idur,
                                  pulsetype='anodicfirst',
                                  delay=10.0 / 1000)
    return gcr.model_cascade(stim)

In [14]:
def run_jensen2007_exp1(model, isis):
    pdur = 1.0 / 1000
    idur = 0.5 / 1000
    num_pulses = 2
    
    gcr = model['gcr']
    tsample = gcr.tsample
    
    exp1 = dict()
    exp1['isis'] = isis
    exp1['freqs'] = 1000.0 / np.array(isis)
    
    # Need to find threshold current first
    amp_search = [0.1, 1.0, 5.0, 10.0, 50.0]
    resp_search = []
    for amp in amp_search:
        mov = run_jensen2007_trial(gcr, amp=amp, freq=1.0, num_pulses=1)
        resp_search.append(mov.data.max())
    amp_th = fit_amps(amp_search, resp_search, 1.0)
    if np.isnan(amp_th) or np.isinf(amp_th):
        exp1['amp_th'] = np.inf
        exp1['amp2amp1'] = 0
        return exp1
    
    exp1['amp_th'] = amp_th
    
    amp2amp1 = []
    for freq in exp1['freqs']:
        cond = '%.1f' % freq
        exp1[cond] = dict()
        
        mov = run_jensen2007_trial(gcr, amp=1.5 * amp_th, freq=freq, num_pulses=num_pulses)
        exp1[cond]['gcr'] = gcr
        exp1[cond]['mov'] = mov
        resp = np.squeeze(mov.data)
        freq_size = int(resp.size / float(num_pulses))
        peaks = []
        for f in range(num_pulses):
            slc = resp[f*freq_size: (f + 1) * freq_size]
            peaks.append(slc.max())

        exp1[cond]['resp'] = np.array(peaks)
        amp2amp1.append(peaks[1] / (peaks[0] + np.finfo(float).eps))
    exp1['amp2amp1'] = np.array(amp2amp1)
        
    return exp1

In [15]:
def run_jensen2007_exp2(model, freqs):
    pdur = 1.0 / 1000
    idur = 0.5 / 1000
    num_pulses = 10
    
    gcr = model['gcr']
    tsample = gcr.tsample

    exp2 = dict()
    exp2['freqs'] = freqs
    
    # Need to find threshold current first
    amp_search = [0.1, 1.0, 5.0, 10.0, 50.0]
    resp_search = []
    for amp in amp_search:
        mov = run_jensen2007_trial(gcr, amp=amp, freq=1.0, num_pulses=1)
        resp_search.append(mov.data.max())
    amp_th = fit_amps(amp_search, resp_search, 1.0)
    exp2['amp_th'] = amp_th
    
    for freq in freqs:
        cond = '%.1f' % freq
        exp2[cond] = dict()
        
        if np.isnan(amp_th) or np.isinf(amp_th):
            exp2[cond]['resp'] = np.zeros(num_pulses)
            exp2[cond]['norm_resp'] = np.zeros(num_pulses)
            continue
        
        mov = run_jensen2007_trial(gcr, amp=1.5 * amp_th, freq=freq, num_pulses=num_pulses)
        exp2[cond]['gcr'] = gcr
        exp2[cond]['mov'] = mov
        resp = np.squeeze(mov.data)
        freq_size = int(resp.size / float(num_pulses))
        peaks = []
        for f in range(num_pulses):
            slc = resp[f*freq_size: (f + 1) * freq_size]
            peaks.append(slc.max())

        exp2[cond]['resp'] = np.array(peaks)
        exp2[cond]['norm_resp'] = np.array(peaks) / (peaks[0] + np.finfo(float).eps)
        
    return exp2

In [16]:
def run_cai2011_exp1(model, amp_th, freqs):
    pdur = 0.2 / 1000
    
    gcr = model['gcr']
    tsample = gcr.tsample

    exp1 = dict()
    exp1['freqs'] = freqs
    exp1['amps'] = np.arange(9) * amp_th
    for freq in freqs:
        cond = '%.0f' % freq
        exp1[cond] = dict()
        
        idur = 0.5 / freq
        
        resp = []
        for amp in exp1['amps']:
            stim = e2cm.Psycho2Pulsetrain(tsample, freq=freq, amp=amp, dur=0.5,
                                          pulse_dur=pdur, interphase_dur=idur,
                                          pulsetype='cathodicfirst',
                                          delay=0.0 / 1000)
            mov = gcr.model_cascade(stim)
            resp.append(np.squeeze(mov.data.max()))

        exp1[cond]['resp'] = np.array(resp)
        exp1[cond]['norm_resp'] = np.array(resp) / 100.0
        
    return exp1

In [17]:
def mse_jensen2005_fig3(x, jensen2005):
    mse = 0
    for diam in [125.0, 500.0]:
        ename = 'E%.0f' % diam
        model = get_model(x, diam)
        fig3 = run_jensen2005_fig3(model, 'SL', jensen2005[ename]['SL'][0, 1])
        if np.any(np.isinf(fig3['amp_match'])):
            return 1e9
        model_sl = fig3['amp_match'] / (fig3['amp_match'][0] + np.finfo(float).eps)
        jensen_sl = jensen2005[ename]['SL'][:, 1] / jensen2005[ename]['SL'][0, 1]
        mse += np.sum((model_sl - jensen_sl) ** 2)

        fig3 = run_jensen2005_fig3(model, 'LL', jensen2005[ename]['LL'][0, 1])
        if np.any(np.isinf(fig3['amp_match'])):
            return 1e9
        model_ll = fig3['amp_match'] / (fig3['amp_match'][0] + np.finfo(float).eps)
        jensen_ll = jensen2005[ename]['LL'][:, 1] / jensen2005[ename]['LL'][0, 1]
        mse += np.sum((model_ll - jensen_ll) ** 2)

    return mse

In [18]:
def mse_jensen2005_fig4(x, jensen2005):
    mse = 0
    for diam in [125.0, 500.0]:
        ename = 'E%.0f' % diam
        for ptype in ['cathodic', 'anodic']:
            model = get_model(x, diam)
            fig4 = run_jensen2005_fig4(model, ptype, jensen2005[ename][ptype][0, 1])
            if np.any(np.isinf(fig4['amp_match'])):
                return 1e9
            model_resp = fig4['amp_match'] / (fig4['amp_match'][0] + np.finfo(float).eps)
            jensen_resp = jensen2005[ename][ptype][:, 1] / jensen2005[ename][ptype][0, 1]
            mse += np.sum((model_resp - jensen_resp) ** 2)

    return mse

In [19]:
def mse_jensen2007_exp1(x, jensen2007, isis=[15, 25, 40, 65, 100, 150, 250, 400, 650]):
    model = get_model(x, 40.0)
    exp1 = run_jensen2007_exp1(model, np.array(isis))
    return np.mean(np.abs(exp1['amp2amp1'] - jensen2007['amp2amp1'][:, 1]) ** 2)

In [20]:
def mse_jensen2007_exp2(x, jensen2007, freqs=[1.5, 2.5, 4, 6.5, 10, 15, 25, 40, 65]):
    model = get_model(x, 40.0)
    exp2 = run_jensen2007_exp2(model, np.array(freqs))
    
    mse = 0
    for freq in exp2['freqs']:
        cond = '%.1f' % freq
        mse += np.mean(np.abs(exp2[cond]['norm_resp'] - jensen2007['norm_resp'][cond][:, 1]) ** 2)
        
#         if exp2[cond]['resp'].max() < 0.5 or np.sum(exp2[cond]['resp']) < exp2[cond]['resp'].size / 2.0:
#             # prevent from zeroing out
#             mse += 1.0 / (np.sum(exp2[cond]['resp']) + np.finfo(float).eps)

    return mse

In [21]:
def mse_cai2011_exp1(x, cai2011, freqs=[100.0, 200.0, 300.0]):
    model = get_model(x, 40.0)
    exp1 = run_cai2011_exp1(model, 20.0, freqs)
    
    mse = 0
    for freq in exp1['freqs']:
        cond = '%.0f' % freq
        
        mse += np.mean(np.abs(exp1[cond]['norm_resp'] - cai2011[cond][:, 1]) ** 2)
    return mse

In [22]:
def mse_all(x, jensen2005, jensen2007, cai2011):
    mse = 0
    
    mse += mse_jensen2005_fig3(x, jensen2005) * 10.0
    mse += mse_jensen2005_fig4(x, jensen2005) * 10.0
    
    mse += mse_jensen2007_exp1(x, jensen2007) * 10.0
    mse += mse_jensen2007_exp2(x, jensen2007)
    
    mse += mse_cai2011_exp1(x, cai2011) * 10.0
    
    print('%s: %.9f' % (str([('%.9f' % xx) for xx in x]), mse))
    return mse

In [ ]:
def get_model(x, diam):
    tsample = 0.005 / 1000

    [ca_scale, inl_scale, anodic_scale, sig_a, sig_b] = x
    model = dict()
    
    # Stimulating electrode remained 25um above the inner limiting membrane
    model['implant'] = e2cm.ElectrodeArray('epiretinal', diam / 2, -500, -500, 0.0)

    r_sampling = 100
    axon_lambda = 2 
    model['retina'] = e2cm.Retina(sampling=r_sampling, axon_lambda=axon_lambda, rot=0,
                                  xlo=model['implant'][0].x_center,
                                  xhi=model['implant'][0].x_center,
                                  ylo=model['implant'][0].y_center,
                                  yhi=model['implant'][0].y_center)

    model['gcr'] = GanglionCell(tsample, ca_scale=ca_scale,
                                inl_scale=inl_scale,
                                anodic_scale=anodic_scale,
                                sig_a=sig_a, sig_b=sig_b)
    return model

In [ ]:
# Version where charge accumulation depends on input pulse

# Good starting points:
# x: array([0.0236761863, 0.112126575, 0.0009215, 49.6662146, 0.363090009, 0.3, 4.51382772])
res = minimize(mse_all, [0.112126575, 49.6662146, 0.363090009, 0.3, 4.51382772], args=(jensen2005, jensen2007, cai2011),
               method='L-BFGS-B', bounds=[(0.01, 20.0), (1.0, 50.0),
                                          (0.01, 0.5),
                                          (0.2, 0.8), (-10.0, 10.0)])

['0.112126575', '49.666214600', '0.363090009', '0.300000000', '4.513827720']: 12.528564600
['0.112126585', '49.666214600', '0.363090009', '0.300000000', '4.513827720']: 12.528564596


# Plot
----------

In [ ]:
print(res)

In [ ]:
model = get_model(res.x, 40.0)

test_freqs = [100, 200, 300]
exp1 = run_cai2011_exp1(model, 20.0, test_freqs)

In [ ]:
num_cols = np.floor(np.sqrt(len(test_freqs)))
num_rows = np.ceil(len(test_freqs) / num_cols)
plt.figure(figsize=(14, 6))
for i, freq in enumerate(test_freqs):
    freqstr = str(freq)
    plt.subplot(num_rows, num_cols, i + 1)
    if 'norm_resp' in exp1[freqstr]:
        plt.plot(exp1[freqstr]['norm_resp'], 'o-b',
                 linewidth=3, label='model')
    if freqstr in cai2011:
        plt.plot(cai2011[freqstr][:, 0],
                 cai2011[freqstr][:, 1], 's-r',
                 linewidth=3, label='rabbit')
    plt.title(freqstr)
#     plt.ylim(0, 1.2)
    plt.xlim(-1, 10)
    plt.legend(loc='best')

In [ ]:
exp1_isis = [15, 25, 40, 65, 100, 150, 250, 400, 650]
model = get_model(res.x, 40.0)
exp1 = run_jensen2007_exp1(model, exp1_isis)
plt.plot(jensen2007['amp2amp1'][:, 0], jensen2007['amp2amp1'][:, 1], 'o-', linewidth=3, label='rabbit')
plt.plot(jensen2007['amp2amp1'][:, 0], exp1['amp2amp1'], 'o-', linewidth=3, label='model')
plt.xlabel('frequency (Hz)')
plt.ylabel('Amp2/Amp1')
plt.title('Mean paired-pulse depression')
plt.legend()

In [ ]:
exp2_freqs = [1.5, 2.5, 4, 6.5, 10, 15, 25, 40, 65]
exp2 = run_jensen2007_exp2(model, exp2_freqs)
num_cols = np.floor(np.sqrt(len(exp2_freqs)))
num_rows = np.ceil(len(exp2_freqs) / num_cols)
plt.figure(figsize=(16, 9))
for i, freq in enumerate(exp2_freqs):
    cond = '%.1f' % freq
    plt.subplot(num_rows, num_cols, i + 1)
    if 'norm_resp' in exp2[cond]:
        plt.plot(exp2[cond]['norm_resp'], 'o-b',
                 linewidth=3, label='model')
    if cond in jensen2007['norm_resp']:
        plt.plot(jensen2007['norm_resp'][cond][:, 0] - 1,
                 jensen2007['norm_resp'][cond][:, 1], 's-r',
                 linewidth=3, label='rabbit')
    plt.title(cond)
    plt.ylim(0, 1.2)
    plt.xlim(-1, 10)
    plt.legend(loc='best')

In [ ]:
def run_fig4_find_amp(model, ptype, amp_th, pulse_durs, verbose=False):
    stim_dur = 500.0 / 1000
    stim_freq = 2.0
    
    gcr = model['gcr']
    tsample = gcr.tsample
    
    fig4 = dict()
    fig4['pulse_durs'] = pulse_durs
    
    # We need to define some activity value that corresponds to threshold activity.
    # We do this by executing the model at the first data point of the Jensen paper.
    # Whatever activity is produced by these parameters, we'll take as threshold
    # activity, and will try to find the matching current for all subsequent data points
    stim = get_jensen2005_pt(ptype, stim_freq, amp_th, pulse_durs[0], stim_dur, tsample)
    mov = gcr.model_cascade(stim)
    act_match = mov.data.max()
    amp_in = np.array([0.05, 0.1, 0.5, 1.0]) * amp_th
    fig4['amp_in'] = amp_in

    if verbose:
        print('- Threshold=%f' %  act_match)

    amp_match = []
    for pd in pulse_durs:
        cond = '%.3f' % pd
        fig4[cond] = dict()

        if verbose:
            print('- Processing pulse dur %f' % pd)
        act_out = []
        for amp in amp_in:
            stim = get_jensen2005_pt(ptype, stim_freq, amp, pd, stim_dur, tsample)
            idx_onset = stim.data.nonzero()[0][0]
            mov = gcr.model_cascade(stim)
            act_out.append(mov.data.max())
        fig4[cond]['act_out'] = act_out
        amp_match.append(fit_amps(amp_in, act_out, act_match))
        
    fig4['amp_match'] = np.array(amp_match)
    return fig4

In [ ]:
plt.figure(figsize=(14, 8))

subplot = 1
for ptype in ['cathodic', 'anodic']:
    for diam in [125, 500]:
        ename = 'E%.0f' % diam
        model = get_model(res.x, diam)
        plt.subplot(2, 2, subplot)
        
        fig4 = run_fig4_find_amp(model, ptype, jensen2005[ename][ptype][0, 1],
                                jensen2005[ename][ptype][:, 0])
        plt.loglog(jensen2005[ename][ptype][:, 0], fig4['amp_match'],
                   'o-b', linewidth=2, markersize=12, label='model')
        plt.loglog(jensen2005[ename][ptype][:, 0], jensen2005[ename][ptype][:, 1],
                   's-r', linewidth=4, markersize=12, label='rabbit')
        
        plt.title('%s responses, %s um electrode' % (ptype, ename))
        plt.legend(loc='upper left')
        plt.xlim(0.01 / 1000, 100.0 / 1000)
        plt.ylim(0.01, 100)
        if subplot > 2:
            plt.xticks(np.array([0.01, 0.1, 1, 10, 100]) / 1000, [0.01, 0.1, 1, 10, 100])
            plt.xlabel('Stimulus pulse duration (ms)')
        else:
            plt.xticks([])
        if (subplot - 1) % 2 == 0:
            plt.yticks([0.01, 1, 10, 100])
            plt.ylabel('Threshold current (uA)')
        else:
            plt.yticks([])
        
        subplot += 1

In [ ]:
def run_fig3_find_amp(model, rtype, amp_th, pdurs):
    stim_dur = 500.0 / 1000
    stim_freq = 2.0
    
    gcr = model['gcr']
    tsample = gcr.tsample
    
    fig3 = dict()
    fig3['pdurs'] = pdurs
    
    # We need to define some activity value that corresponds to threshold activity.
    # We do this by executing the model at the first data point of the Jensen paper.
    # Whatever activity is produced by these parameters, we'll take as threshold
    # activity, and will try to find the matching current for all subsequent data points
    stim = get_jensen2005_pt('cathodic', stim_freq, amp_th, pdurs[0], stim_dur, tsample)
    mov = gcr.model_cascade(stim)
    sl_max, ll_max = find_sl_ll(mov, stim, gcr.delay_inl)
    if rtype == 'SL':
        act_match = sl_max[1]
    else:
        act_match = ll_max[1]

    amp_in = np.array([0.05, 0.1, 0.5, 1.0]) * amp_th
    fig3['amp_in'] = amp_in
    fig3['act_match'] = act_match

    amp_match = []
    for pd in pdurs:
        cond = '%.3f' % pd
        fig3[cond] = dict()

        act_out = []
        for amp in amp_in:
            stim = get_jensen2005_pt('cathodic', stim_freq, amp, pd, stim_dur, tsample)
            mov = gcr.model_cascade(stim)
            sl_max, ll_max = find_sl_ll(mov, stim, gcr.delay_inl)
            if rtype == 'SL':
                act_out.append(sl_max[1])
            else:
                act_out.append(ll_max[1])
        fig3[cond]['act_out'] = act_out
        amp_match.append(fit_amps(amp_in, act_out, act_match))
        
    fig3['amp_match'] = np.array(amp_match)
    return fig3

In [ ]:
plt.figure(figsize=(14, 8))

subplot = 1
for rtype in ['SL', 'LL']:
    for diam in [125, 500]:
        ename = 'E%.0f' % diam
        model = get_model(res.x, diam)
        plt.subplot(2, 2, subplot)
        
        fig3 = run_fig3_find_amp(model, rtype, jensen2005[ename][rtype][0, 1],
                                jensen2005[ename][rtype][:, 0])
        plt.loglog(jensen2005[ename][ptype][:, 0], fig3['amp_match'],
                   'o-b', linewidth=2, markersize=12, label='model')
        plt.loglog(jensen2005[ename][rtype][:, 0], jensen2005[ename][rtype][:, 1],
                   's-r', linewidth=4, markersize=12, label='rabbit')
        
        plt.title('%s responses, %s um electrode' % (rtype, ename))
        plt.legend(loc='upper left')
        plt.xlim(0.01 / 1000, 100.0 / 1000)
        plt.ylim(0.01, 100)
        if subplot > 2:
            plt.xticks(np.array([0.01, 0.1, 1, 10, 100]) / 1000, [0.01, 0.1, 1, 10, 100])
            plt.xlabel('Stimulus pulse duration (ms)')
        else:
            plt.xticks([])
        if (subplot - 1) % 2 == 0:
            plt.yticks([0.01, 1, 10, 100])
            plt.ylabel('Threshold current (uA)')
        else:
            plt.yticks([])
        
        subplot += 1